# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [55]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point

# Import API key
from api_keys import geoapify_key

In [56]:
# Check if the API key is not empty
if geoapify_key:
    url = f'https://api.geoapify.com/v1/ipinfo?apiKey={geoapify_key}'

    try:
        # Make the API request
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            data = response.json()
            print("API Key is working. Response:")
            print(data)
        else:
            print(f"Error: {response.status_code}")
    except Exception as e:
        print(f"Error: {str(e)}")
else:
    print("Error: Geoapify API key is empty.")


API Key is working. Response:
{'city': {'names': {'en': 'Toronto (North York)'}, 'name': 'Toronto (North York)'}, 'continent': {'code': 'NA', 'geoname_id': 6255149, 'names': {'de': 'Nordamerika', 'en': 'North America', 'es': 'Norteamérica', 'fa': ' امریکای شمالی', 'fr': 'Amérique Du Nord', 'ja': '北アメリカ大陸', 'ko': '북아메리카', 'pt-BR': 'América Do Norte', 'ru': 'Северная Америка', 'zh-CN': '北美洲'}, 'name': 'North America'}, 'country': {'geoname_id': 6251999, 'iso_code': 'CA', 'names': {'de': 'Kanada', 'en': 'Canada', 'es': 'Canadá', 'fa': 'کانادا', 'fr': 'Canada', 'ja': 'カナダ', 'ko': '캐나다', 'pt-BR': 'Canadá', 'ru': 'Канада', 'zh-CN': '加拿大'}, 'name': 'Canada', 'name_native': 'Canada', 'phone_code': '1', 'capital': 'Ottawa', 'currency': 'CAD', 'flag': '🇨🇦', 'languages': [{'iso_code': 'en', 'name': 'English', 'name_native': 'English'}, {'iso_code': 'fr', 'name': 'French', 'name_native': 'Français'}]}, 'location': {'latitude': 43.7687, 'longitude': -79.4109}, 'subdivisions': [{'names': {'en': 'Ont

In [57]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
file_path = r"C:\Users\Qazi Fabia Hoq\OneDrive\Desktop\python-api-challenge\output_data\cities.csv"
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san antonio de pale,-1.4014,5.6325,26.80,80,100,4.15,GQ,1701479575
1,1,albany,42.6001,-73.9662,6.79,88,100,0.89,US,1701479324
2,2,la asuncion,11.0333,-63.8628,26.87,85,6,7.79,VE,1701479575
3,3,aswan,24.0934,32.9070,18.03,33,0,3.94,EG,1701479575
4,4,makinsk,52.6292,70.4169,-2.06,96,100,6.50,KZ,1701479575


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [74]:
try:
    import geopandas
    print("Geopandas version:", geopandas.__version__)
except ImportError:
    print("Geopandas is not installed.")

try:
    import geoviews
    print("Geoviews version:", geoviews.__version__)
except ImportError:
    print("Geoviews is not installed.")

try:
    import hvplot
    print("hvplot version:", hvplot.__version__)
except ImportError:
    print("hvplot is not installed.")

try:
    import requests
    print("requests version:", requests.__version__)
except ImportError:
    print("requests is not installed.")
try:
    import folium
    print("Folium version:", folium.__version__)
except ImportError:
    print("Folium is not installed.")

try:
    import matplotlib
    print("Matplotlib version:", matplotlib.__version__)
except ImportError:
    print("Matplotlib is not installed.")

try:
    import plotly
    print("Plotly version:", plotly.__version__)
except ImportError:
    print("Plotly is not installed.")

Geopandas version: 0.14.1
Geoviews version: 1.11.0
hvplot version: 0.8.4
requests version: 2.31.0
Folium version: 0.15.0
Matplotlib version: 3.7.2
Plotly version: 5.9.0


In [66]:
print(geo_df.info())
# or
print(geo_df.dtypes)


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   City_ID     556 non-null    int64   
 1   City        556 non-null    object  
 2   Lat         556 non-null    float64 
 3   Lng         556 non-null    float64 
 4   Max Temp    556 non-null    float64 
 5   Humidity    556 non-null    int64   
 6   Cloudiness  556 non-null    int64   
 7   Wind Speed  556 non-null    float64 
 8   Country     553 non-null    object  
 9   Date        556 non-null    int64   
 10  geometry    556 non-null    geometry
dtypes: float64(4), geometry(1), int64(4), object(2)
memory usage: 47.9+ KB
None
City_ID          int64
City            object
Lat            float64
Lng            float64
Max Temp       float64
Humidity         int64
Cloudiness       int64
Wind Speed     float64
Country         object
Date             int64
geometry      geometry
dtype: 

In [87]:
import folium
from folium.plugins import MarkerCluster
from branca.colormap import linear

# Create a base map centered around the first city with English labels
m = folium.Map(location=[city_data_df['Lat'].iloc[0], city_data_df['Lng'].iloc[0]], zoom_start=5, control_scale=True)

# Create a colormap for humidity
colormap = linear.YlOrRd_09.scale(city_data_df['Humidity'].min(), city_data_df['Humidity'].max())

# Add markers for each city
marker_cluster = MarkerCluster().add_to(m)

for index, row in city_data_df.iterrows():
    # Adjust radius based on humidity
    radius = row['Humidity'] / 10
    # Get color from the colormap
    color = colormap(row['Humidity'])
    
    # Create a tooltip with city information
    tooltip = f"{row['City']} - Humidity: {row['Humidity']}"
    
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=tooltip,
    ).add_to(marker_cluster)

# Add the colormap to the map
colormap.caption = 'Humidity'
m.add_child(colormap)

# Display the map
m


In [80]:
c='Humidity',
print(geo_df.head())


   City_ID                 City      Lat      Lng  Max Temp  Humidity  \
0        0  san antonio de pale  -1.4014   5.6325     26.80        80   
1        1               albany  42.6001 -73.9662      6.79        88   
2        2          la asuncion  11.0333 -63.8628     26.87        85   
3        3                aswan  24.0934  32.9070     18.03        33   
4        4              makinsk  52.6292  70.4169     -2.06        96   

   Cloudiness  Wind Speed Country        Date                    geometry  
0         100        4.15      GQ  1701479575    POINT (5.63250 -1.40140)  
1         100        0.89      US  1701479324  POINT (-73.96620 42.60010)  
2           6        7.79      VE  1701479575  POINT (-63.86280 11.03330)  
3           0        3.94      EG  1701479575   POINT (32.90700 24.09340)  
4         100        6.50      KZ  1701479575   POINT (70.41690 52.62920)  


In [83]:
geo_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,geometry
0,0,san antonio de pale,-1.4014,5.6325,26.80,80,100,4.15,GQ,1701479575,POINT (5.63250 -1.40140)
1,1,albany,42.6001,-73.9662,6.79,88,100,0.89,US,1701479324,POINT (-73.96620 42.60010)
2,2,la asuncion,11.0333,-63.8628,26.87,85,6,7.79,VE,1701479575,POINT (-63.86280 11.03330)
3,3,aswan,24.0934,32.9070,18.03,33,0,3.94,EG,1701479575,POINT (32.90700 24.09340)
4,4,makinsk,52.6292,70.4169,-2.06,96,100,6.50,KZ,1701479575,POINT (70.41690 52.62920)


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [88]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_conditions = (
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
)

# Filter the DataFrame
ideal_cities_df = city_data_df[ideal_weather_conditions]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,wailua homesteads,22.0669,-159.3780,26.24,87,0,3.60,US,1701479579
96,96,diu,20.7141,70.9822,21.94,54,0,4.40,IN,1701479596
391,391,lihue,21.9789,-159.3672,26.58,83,0,3.60,US,1701479669
439,439,kapa'a,22.0752,-159.3190,26.99,86,0,3.60,US,1701479684
454,454,jeddah,21.5169,39.2192,25.92,78,0,1.03,SA,1701479390
482,482,mermoz boabab,14.7065,-17.4758,24.88,88,0,3.09,SN,1701479693
505,505,remire-montjoly,4.9167,-52.2667,25.02,88,0,0.00,GF,1701479697
545,545,rufino,-34.2683,-62.7126,21.11,61,0,3.14,AR,1701479708


### Step 3: Create a new DataFrame called `hotel_df`.

In [89]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
28,wailua homesteads,US,22.0669,-159.3780,87,
96,diu,IN,20.7141,70.9822,54,
391,lihue,US,21.9789,-159.3672,83,
439,kapa'a,US,22.0752,-159.3190,86,
454,jeddah,SA,21.5169,39.2192,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [86]:
# Set parameters to search for hotels
radius = 100000  # Updated radius to 100,000 meters
params = {
    'radius': radius,
    'type': 'hotel',
    'limit': 10,  # Increase the limit to find more hotels per city
    'sort': 'distance',
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"proximity({lng},{lat},{radius})"
    params["bias"] = f"proximity({lng},{lat})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the hotels from the results and store the names in the hotel_df DataFrame
    hotel_names = []
    try:
        for feature in name_address["features"]:
            hotel_names.append(feature["properties"]["name"])
    except (KeyError, IndexError):
        # If no hotels are found, set the hotel name as "No hotel found".
        hotel_names.append("No hotel found")
        
    # Join the hotel names into a comma-separated string
    hotel_df.loc[index, "Hotel Name"] = ', '.join(hotel_names)
    
    # Log the search results
    num_hotels = len(hotel_names)
    print(f"{hotel_df.loc[index, 'City']} - {num_hotels} hotels found: {', '.join(hotel_names)}")

# Display the entire hotel_df
hotel_df


Starting hotel search
wailua homesteads - 1 hotels found: No hotel found
diu - 1 hotels found: No hotel found
lihue - 1 hotels found: No hotel found
kapa'a - 1 hotels found: No hotel found
jeddah - 1 hotels found: No hotel found
mermoz boabab - 1 hotels found: No hotel found
remire-montjoly - 1 hotels found: No hotel found
rufino - 1 hotels found: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
28,wailua homesteads,US,22.0669,-159.3780,87,No hotel found
96,diu,IN,20.7141,70.9822,54,No hotel found
391,lihue,US,21.9789,-159.3672,83,No hotel found
439,kapa'a,US,22.0752,-159.3190,86,No hotel found
454,jeddah,SA,21.5169,39.2192,78,No hotel found
482,mermoz boabab,SN,14.7065,-17.4758,88,No hotel found
505,remire-montjoly,GF,4.9167,-52.2667,88,No hotel found
545,rufino,AR,-34.2683,-62.7126,61,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [94]:
import folium
from folium.plugins import MarkerCluster
from branca.colormap import linear

# Create a base map centered around the first city with English labels
m = folium.Map(location=[city_data_df['Lat'].mean(), city_data_df['Lng'].mean()], zoom_start=5, control_scale=True)

# Create a colormap for humidity
colormap = linear.YlOrRd_09.scale(city_data_df['Humidity'].min(), city_data_df['Humidity'].max())

# Add markers for each city
marker_cluster = MarkerCluster().add_to(m)

for row in city_data_df.itertuples():
    # Adjust radius based on humidity
    radius = row.Humidity / 10
    # Get color from the colormap
    color = colormap(row.Humidity)
    
    # Check if hotel information is available for the current city
    if row.Index in hotel_df.index:
        # Access the hotel information
        hotel_name = hotel_df.loc[row.Index, 'Hotel Name']
    else:
        hotel_name = "No hotel found"
    
    # Create a tooltip with city information, hotel name, and country
    tooltip = f"{row.City} - Humidity: {row.Humidity}<br>Hotel: {hotel_name}<br>Country: {row.Country}"
    
    folium.CircleMarker(
        location=[row.Lat, row.Lng],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=tooltip,
    ).add_to(marker_cluster)

# Add the colormap to the map
colormap.caption = 'Humidity'
m.add_child(colormap)

# Display the map
m
